## Algoritmia <font color="red"> NOTA: 10
### Práctica 1
El objetivo de esta práctica es trabajar con iteradores y generadores.

Se pide la implementación de las funciones que aparecen a continuación. 

En el cuerpo de cada función hay una instrucción "pass", se debe sustituir por la implementación adecuada. 

Para cada función que se pide se proporciona una función con algunos tests. 

Al llamar a las funciones de test no debería saltar ninguna aserción.

### `iterador_con_sustitucion`   <font color="red"> OK

In [9]:
"""
    @Author : Diego Sanz Villafruela
    
    Dado un iterable genera sus valores una vez aplicadas las sustituciones 
    indicadas por el diccionario de cambios.
    Los valores no hay que devolverlos todos a la vez, se deben generar de uno 
    en uno.
    
    @param iterable: elemento iterable
    @param cambios: mapa 
    """
def iterador_con_sustitucion(iterable, cambios):
    
    for elemento in iterable:
        # si el elemento se encuentra en el mapa, devolvemos el valor asociado
        if elemento in cambios:
            yield cambios[elemento]
        # si no se encuentra en el mapa devolvemos el elemento
        else:
            yield elemento
   
        

In [10]:
def test_iterador_con_sustitucion(): 
    """
    Casos de prueba para iterador_con_sustitucion().
    """
    
    for iterable, cambios, iterable_sustituido in (
        ([1, 2, 3, 4, 1, 2], {2: 1, 1: 2, 3: 5}, [2, 1, 5, 4, 2, 1]),
        ([1, 2, 3, 4, 1, 2] * 100, {2: 1, 1: 2, 3: 5}, 
            [2, 1, 5, 4, 2, 1] * 100),
        ("abcdb" * 100, {'a': 'z', 'b': 'a', 'd': 'y'},
            ['z', 'a', 'c', 'y', 'a'] * 100)
    ):
        assert (list(iterador_con_sustitucion(iterable, cambios)) 
                == iterable_sustituido)
        it = iterador_con_sustitucion(iterable, cambios)
        for e in iterable_sustituido:
            assert e == next(it)
            
    for v in iterador_con_sustitucion(range(10**100), {0: 0}):
        if v >= 100:
            break
            
    return True
            
if __name__ == "__main__": 
    test_iterador_con_sustitucion()
    print("OK")  

OK


### `iterador_anidado`  <font color="red"> Ok

In [11]:
import collections   # por si es necesario usar collections.Iterable

"""
    @Author : Diego Sanz Villafruela

    Iterador que genera los valores en elemento recursivamente: si elemento no 
    es iterable genera solo elemento, pero si elemento es iterable genera sus
    elementos de manera recursiva.
    Los valores se deben generar de uno en uno.
    
    @param elemento: elemento iterable ej: [1,2,3,[4,5,6] ,[1,[1,2] ] ]
    """
def iterador_anidado(elemento):
    # comprobamos si el elemento es itererable
    if not isinstance(elemento, collections.Iterable):
        yield elemento;
    else:
        # si el elemento es iterable
        for e in elemento:
            # recorremos el iterador
            for i in iterador_anidado(e):
                yield i
    

In [12]:
def test_iterador_anidado():
    """
    Casos de prueba para iterador_anidado()
    """
    
    assert isinstance([4], collections.Iterable)

    assert not isinstance(4, collections.Iterable)
    
    assert list(iterador_anidado(4)) == [4]

    assert list(iterador_anidado([4])) == [4]

    assert list(iterador_anidado((4,))) == [4]

    assert list(iterador_anidado([[4]])) == [4]

    assert list(iterador_anidado([1, [2, [3], 4]])) == [1, 2, 3, 4]

    l1 = []; l2 = []; l3 = []
    for i in range(100):
        l1 += [i]
        l2 = [l2, i]
        assert l1 == list(iterador_anidado(l2))
        l3 = [(l3, [i])]
        assert l1 == list(iterador_anidado(l3))
        
    for v in iterador_anidado(range(10**100)):
        if v > 100:
            break
    
    return True

if __name__ == "__main__": 
    test_iterador_anidado()
    print("OK")

OK


### `generador_media_movil` <font color="red"> OK

In [13]:
"""
    @Author : Diego Sanz Villafruela
    
    Este metodo esta hecho para que sea muy eficiente. En vez de volver ha sumar todos los elementos de la secuencia,
    se resta el antiguo y se suma el nuevo. 
    
    Así conseguimos O(len(iterable)), de la otra forma sería O(len(iterable) * longitud)

    Dado un iterable de valores numéricos, genera los valores de la media móvil 
    de la longitud indicada.
    Por ejemplo, si la longitud es 3, generaría la media de los 3 primeros
    valores, de los valores del 2º al 4º, de los valores del 3º al 5º...
    Los valores se deben generar de uno en uno.
    
    @param iterable: un iterable de valores numericos
    @param longitud: la longitud utilizada para calcular la media movil
    """
# complejidad de O(len(iterable)) muy eficiente
def generador_media_movil(iterable, longitud): 
        
    if iterable == None or longitud < 1:
        raise ValueError('Parametros incorrectos')
    iterador = iter(iterable)
    acumulador = 0
    # uso de una lista de tamano = longitud, 
    # para almacenar elementos y no tener que voler a recorrerlos.
    lista = []

    # calculo inicial de los n longitud elementos
    for i in range(longitud):
        valor = next(iterador,None)
        if valor is not None:
            lista.append(valor)
            acumulador += valor
        else:
            raise ValueError('Error: len(iterable) < longitud')
    
    yield acumulador/longitud
    
    indice = 0;
    acumulador -= lista[indice]
    valor = next(iterador,None)
    
    # mientras siga habiendo elementos
    while valor != None:
        # añadimos el nuevo valor
        lista[indice] = valor
        acumulador += valor
        yield acumulador/longitud
        
        valor = next(iterador,None)
        indice += 1
        
        # borramos el antiguo valor, y reseteamos indice de la lista si es = a longitud
        # la lista tiene tamano longitud, reseteamos el indice cuando llegue a longitud
        if indice  == longitud:
            indice = 0
        acumulador -= lista[indice]

        


In [14]:
def test_generador_media_movil(): 
    """
    Casos de prueba para generador_media_movil().
    """
    
    for secuencia in (list(range(10)), tuple(range(10)), range(10)):
        assert (list(generador_media_movil(secuencia, 1))
                == [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0])
        assert (list(generador_media_movil(secuencia, 2))
                == [0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5])
        assert (list(generador_media_movil(secuencia, 3))
                == [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0])   
        assert (list(generador_media_movil(secuencia, 4)) 
                == [1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5])  
        assert (list(generador_media_movil(secuencia, 5))
                == [2.0, 3.0, 4.0, 5.0, 6.0, 7.0])  

    assert list(generador_media_movil(range(100), 1)) == list(range(100))    
    assert list(generador_media_movil(range(100), 3)) == list(range(1, 99))    
    assert list(generador_media_movil(range(100), 5)) == list(range(2, 98))
    
    assert list(generador_media_movil(range(100), 2)) == [x + 0.5 for x in range(99)]
    assert list(generador_media_movil(range(100), 4)) == [x + 1.5 for x in range(97)]

    assert (list(generador_media_movil(range(100, 0, -1), 1)) 
            == list(range(100, 0, -1)))
    assert (list(generador_media_movil(range(100, 0, -1), 3)) 
            == list(range(99, 1, -1)))
    assert (list(generador_media_movil(range(100, 0, -1), 5)) 
            == list(range(98, 2, -1)))
    
    it = generador_media_movil(range(1000),4)
    for v in range(997):
        assert next(it) == v + 1.5 
        
    assert list(generador_media_movil([1, 2] * 1000, 2)) == [1.5] * 1999     
    assert list(generador_media_movil([1, 2] * 1000, 3)) == [4/3, 5/3] * 999
       
    for v in generador_media_movil(range(10**100), 10):
        if v >= 100:
            break
        
    return True

if __name__ == "__main__": 
    test_generador_media_movil()
    print("OK")

OK


### `iterador_incluido`  <font color="red"> OK

In [15]:
"""
    @Author : Diego Sanz Villafruela

    Dado un primer iterador o iterable, comprueba que sus elementos están
    incluidos en el mismo orden en los elementos de un segundo iterador o 
    iterable.
    
    @param itera_1: iterador o iterable de los elementos a comprobar
    @param itera_2: iterador o iterable de los elementos que han de estar incluidos en el mismo orden
    """
def iterador_incluido(itera_1, itera_2):
    
    it2 = iter(itera_2)
    
    for elem_1 in itera_1:
        valor = next(it2,None)
        # si no quedan mas elementos en itera_2
        if valor is None:
            return False
        # hasta que no encontremos un elemento igual en itera_2
        while valor != elem_1:
            valor = next(it2,None)
            # si no quedan mas elementos en itera_2
            if valor is None:
                return False
    
    return True

In [16]:
def test_iterador_incluido():
    """
    Casos de prueba para iterador_incluido().
    """
    
    assert iterador_incluido(range(100), range(100))     
    assert iterador_incluido(range(99), range(100))     
    assert iterador_incluido(range(1,100), range(100))     

    assert not iterador_incluido(range(100), range(99))
    assert not iterador_incluido(range(100), range(1, 100))
    
    assert iterador_incluido(range(10, 90, 3), range(100))
    assert not iterador_incluido(range(10, 110, 3), range(100))

    assert not iterador_incluido(range(10, 110, 3), range(100))    
    
    l = list(range(10, 90, 3))
    assert iterador_incluido(l, range(100))
    l[20] = 11
    assert not iterador_incluido(l, range(100))
    assert not iterador_incluido(iter(l), range(100))
    
    assert iterador_incluido(range(1000), range(10**100))
    assert not iterador_incluido(range(10**100), range(1000))
    
    return True
    
if __name__ == "__main__": 
    test_iterador_incluido()
    print("OK")    

OK
